In [53]:
import numpy as np
import matplotlib.pyplot as plt
tol=1e-8
eps=1e-6

In [54]:
variables = {
    "P0_S": 341.3,
    "C_C": 0.66,
    "r_SM": 0.1065,
    "r_SC": 0.220,
    "r_SE": 0.170,
    "a_O3": 0.080,
    "a_SC": 0.1239,
    "a_SW": 0.1451,
    "r_LC": 0.195,
    "r_LE": 0.0,
    "a_LC": 0.622,
    "a_LW": 0.8258,
    "eps_E": 1,
    "eps_A": 0.875,
    "f_A": 0.618,
    "alpha": 3,
    "beta": 4,
    "sigma":5.670*10**(-8),
    "r_LA": 0
}


In [60]:
def f(TE, TA, var):
    return np.array([var["eps_E"]*(var["f_A"]*var["eps_A"]*var["sigma"]*TA**4+var["r_LA"]*var["eps_E"]* var["sigma"]*TE**4)/(1-var["r_LA"]*(1-var["eps_E"]))
            +(1-var["r_SE"])*((1-var["a_O3"])*(1-var["a_SW"])*(1-var["r_SM"])*var["P0_S"])/(1-var["r_SM"]*var["r_SE"])-(var["alpha"]+var["beta"])*(TE-TA)-var["eps_E"]*var["sigma"]*TE**4,
            (1-var["r_LA"])*var["a_LW"]*(var["eps_E"] *var["sigma"]* TE**4+(1-var["eps_E"])*var["f_A"]*var["eps_A"]*var["sigma"]*TA**4 )/(1-var["r_LA"]*(1-var["eps_E"]))
            +(var["a_SW"]+var["a_O3"]*(1-var["a_SW"]))*(1-var["r_SM"])*(var["r_SE"]*(1-var["a_O3"])*(1-var["a_SW"])*(1-var["r_SM"])*var["P0_S"])/(1-var["r_SM"]*var["r_SE"])
            +(var["a_O3"]+var["a_SW"]*(1-var["a_O3"])*(1-var["r_SM"]))*var["P0_S"]+(var["alpha"]+var["beta"])*(TE-TA)-var["eps_A"]*var["sigma"]*TA**4])

def fprime(TE, TA, var):
    return np.array([[var["eps_E"]*(4*var["r_LA"]*var["eps_E"]* var["sigma"]*TE**3)/(1-var["r_LA"]*(1-var["eps_E"]))-(var["alpha"]+var["beta"])-4*var["eps_E"]*var["sigma"]*TE**3,
                      var["eps_E"]*(4*var["f_A"]*var["eps_A"]*var["sigma"]*TA**3)/(1-var["r_LA"]*(1-var["eps_E"]))+(var["alpha"]+var["beta"])],[
                      (1-var["r_LA"])*var["a_LW"]*(var["eps_E"] *var["sigma"]*4* TE**3 )/(1-var["r_LA"]*(1-var["eps_E"]))+(var["alpha"]+var["beta"]),
                      (1-var["r_LA"])*var["a_LW"]*((1-var["eps_E"])*var["f_A"]*var["eps_A"]*var["sigma"]*4*TA**3 )/(1-var["r_LA"]*(1-var["eps_E"]))-(var["alpha"]+var["beta"])-4*var["eps_A"]*var["sigma"]*TA**3]])





In [61]:
def solve(TE, TA, var):
    while (sum(abs(f(TE, TA, var)))>tol):
        print(TE, " ", TA, " ", f(TE, TA, var), fprime(TE, TA, var))
        diffs=np.linalg.inv(fprime(TE, TA, var))@f(TE, TA, var)
        TE-=diffs[0]
        TA-=diffs[1]

    return TE, TA

TE, TA=500, 500

solve(TE, TA, variables)

500   500   [-1424.72230341   -98.41636694] [[-35.3501274   22.33039914]
 [ 30.41158447 -31.80638854]]
393.29097418117755   394.8762474395785   [-397.25993302  -21.1589255 ] [[-20.79707951  14.55138983]
 [ 18.39365349 -19.21903233]]
333.1391186808617   336.206732095489   [-82.40604075  -2.71908704] [[-15.38537347  11.66083333]
 [ 13.9246577  -14.54177039]]
313.0920595118088   316.8234303181412   [-7.05726128 -0.12776329] [[-13.96084272  10.9002863 ]
 [ 12.74827785 -13.31111542]]
311.05827451575686   314.86604217299737   [-0.06688516 -0.00059152] [[-13.8260734   10.82844239]
 [ 12.63698513 -13.19486349]]
311.03877764644903   314.84732480112984   [-6.12275176e-06 -6.23009555e-09] [[-13.82478992  10.82775968]
 [ 12.63592523 -13.19375879]]


(311.0387758727391, 314.8473231019404)

In [64]:
def find_derivs(TE_init, TA_init, var):
    TE, TA=solve(TE_init, TA_init, var)
    derivatives={}
    for variable, value in var.items():
        if value==0:
          continue
        print(variable)
        var[variable]=value*(1+eps)
        TE_new, TA_new=solve(TE, TA, var)
        derivatives[variable]=[(TE_new-TE)/(value*eps), (TA_new-TA)/(value*eps)]
        var[variable]=value
    print(derivatives)

find_derivs(0, 1, variables)





0   1   [209.74374283  68.9341285 ] [[-7.000014    7.00001412]
 [ 7.000014   -7.0000142 ]]
3676105280.534129   3676105187.163945   [-4.75536361e+30 -5.09441337e+29] [[-1.12670082e+22  6.09265858e+21]
 [ 9.30431396e+21 -9.85864147e+21]]
2757078960.400597   2757078890.372959   [-1.50462677e+30 -1.61190423e+29] [[-4.75326908e+21  2.57034034e+21]
 [ 3.92525745e+21 -4.15911437e+21]]
2067809220.300448   2067809167.7797194   [-4.76073313e+29 -5.10016573e+28] [[-2.00528539e+21  1.08436233e+21]
 [ 1.65596799e+21 -1.75462637e+21]]
1550856915.225336   1550856875.8347895   [-1.50632572e+29 -1.61372431e+28] [[-8.45979774e+20  4.57465358e+20]
 [ 6.98611495e+20 -7.40233002e+20]]
1163142686.4190023   1163142656.8760924   [-4.76610872e+28 -5.10592458e+27] [[-3.56897717e+20  1.92993198e+20]
 [ 2.94726724e+20 -3.12285798e+20]]
872357014.8142517   872356992.6570694   [-1.50802659e+28 -1.61554645e+27] [[-1.50566224e+20  8.14190054e+19]
 [ 1.24337837e+20 -1.31745571e+20]]
654267761.1106887   654267744.49280

In [75]:
def fcloud(TE, TA, var):
    r_SA=1-(1-var["r_SM"])*(1-var["C_C"]*var["r_SC"])
    a_SA=1-(1-var["a_SW"])*(1-var["C_C"]*var["a_SC"])
    a_LA=1-(1-var["a_LW"])*(1-var["C_C"]*var["a_LC"])
    r_LA=var["C_C"]*var["r_LC"]

    return np.array([var["eps_E"]*(var["f_A"]*var["eps_A"]*var["sigma"]*TA**4+r_LA*var["eps_E"]* var["sigma"]*TE**4)/(1-r_LA*(1-var["eps_E"]))
            +(1-var["r_SE"])*((1-var["a_O3"])*(1-a_SA)*(1-r_SA)*var["P0_S"])/(1-r_SA*var["r_SE"])-(var["alpha"]+var["beta"])*(TE-TA)-var["eps_E"]*var["sigma"]*TE**4,
            (1-r_LA)*a_LA*(var["eps_E"] *var["sigma"]* TE**4+(1-var["eps_E"])*var["f_A"]*var["eps_A"]*var["sigma"]*TA**4 )/(1-r_LA*(1-var["eps_E"]))
            +(a_SA+var["a_O3"]*(1-a_SA))*(1-r_SA)*(var["r_SE"]*(1-var["a_O3"])*(1-a_SA)*(1-r_SA)*var["P0_S"])/(1-r_SA*var["r_SE"])
            +(var["a_O3"]+a_SA*(1-var["a_O3"])*(1-r_SA))*var["P0_S"]+(var["alpha"]+var["beta"])*(TE-TA)-var["eps_A"]*var["sigma"]*TA**4])

def fcloudprime(TE, TA, var):
    a_LA=1-(1-var["a_LW"])*(1-var["C_C"]*var["a_LC"])
    r_LA=var["C_C"]*var["r_LC"]
    return np.array([[var["eps_E"]*(4*r_LA*var["eps_E"]* var["sigma"]*TE**3)/(1-r_LA*(1-var["eps_E"]))-(var["alpha"]+var["beta"])-4*var["eps_E"]*var["sigma"]*TE**3,
                      var["eps_E"]*(4*var["f_A"]*var["eps_A"]*var["sigma"]*TA**3)/(1-r_LA*(1-var["eps_E"]))+(var["alpha"]+var["beta"])],[
                      (1-r_LA)*a_LA*(var["eps_E"] *var["sigma"]*4* TE**3 )/(1-r_LA*(1-var["eps_E"]))+(var["alpha"]+var["beta"]),
                      (1-r_LA)*a_LA*((1-var["eps_E"])*var["f_A"]*var["eps_A"]*var["sigma"]*4*TA**3 )/(1-r_LA*(1-var["eps_E"]))-(var["alpha"]+var["beta"])-4*var["eps_A"]*var["sigma"]*TA**3]])



In [76]:
def solvecloud(TE, TA, var):
    while (sum(abs(fcloud(TE, TA, var)))>tol):
        print(TE, " ", TA, " ", fcloud(TE, TA, var), fcloudprime(TE, TA, var))
        diffs=np.linalg.inv(fcloudprime(TE, TA, var))@fcloud(TE, TA, var)
        TE-=diffs[0]
        TA-=diffs[1]

    return TE, TA

TE, TA=500, 500

solvecloud(TE, TA, variables)

500   500   [-1008.59157403  -244.23743067] [[-31.70144685  22.3303952 ]
 [ 29.16495166 -31.8063872 ]]
394.87678031592714   395.9281030487029   [-277.56510006  -62.75960142] [[-19.16738986  14.61189392]
 [ 17.9179688  -19.3169376 ]]
336.9760509383108   338.9717113772795   [-55.45693124 -11.43580284] [[-14.56153639  11.77677276]
 [ 13.78507272 -14.72937565]]
318.72620273844507   321.1154699732045   [-4.3049601 -0.8267548] [[-13.39832637  11.0609545 ]
 [ 12.74130827 -13.57109689]]
317.07413781543386   319.5034983366358   [-0.0327748  -0.00600146] [[-13.29934739  11.00010429]
 [ 12.65249305 -13.47263404]]
317.0614478495886   319.4911354119236   [-1.9285892e-06 -3.3832498e-07] [[-13.29859108  10.99963997]
 [ 12.65181441 -13.47188271]]


(317.06144710685413, 319.4911346892879)

In [77]:
def find_derivs(TE_init, TA_init, var):
    TE, TA=solvecloud(TE_init, TA_init, var)
    derivatives={}
    for variable, value in var.items():
        if value==0:
          continue
        print(variable)
        var[variable]=value*(1+eps)
        TE_new, TA_new=solvecloud(TE, TA, var)
        derivatives[variable]=[(TE_new-TE)/(value*eps), (TA_new-TA)/(value*eps)]
        var[variable]=value
    print(derivatives)

find_derivs(0, 1, variables)


0   1   [169.78989703  78.94199781] [[-7.000014    7.00001412]
 [ 7.000014   -7.0000142 ]]
3281079943.3419375   3281079862.600453   [-2.17212916e+30 -6.12261371e+29] [[-6.98011486e+21  4.33204885e+21]
 [ 6.26335370e+21 -7.00976832e+21]]
2460809957.5064535   2460809896.95034   [-6.87275242e+29 -1.93723324e+29] [[-2.94473596e+21  1.82758311e+21]
 [ 2.64235234e+21 -2.95724601e+21]]
1845607468.1298404   1845607422.7127547   [-2.17458182e+29 -6.12952706e+28] [[-1.24231048e+21  7.71011624e+20]
 [ 1.11474239e+21 -1.24758816e+21]]
1384205601.0973797   1384205567.0345654   [-6.88051279e+28 -1.93942067e+28] [[-5.24099735e+20  3.25270529e+20]
 [ 4.70281947e+20 -5.26326255e+20]]
1038154200.8230348   1038154175.2759242   [-2.17703725e+28 -6.13644822e+27] [[-2.21104576e+20  1.37223504e+20]
 [ 1.98400197e+20 -2.22043889e+20]]
778615650.6172761   778615631.456943   [-6.88828192e+27 -1.94161057e+27] [[-9.32784928e+19  5.78911659e+19]
 [ 8.37000829e+19 -9.36747656e+19]]
583961737.962957   583961723.5927